# Computer Vision Taken

Computer vision omvat een breed scala aan taken die elk specifieke uitdagingen en toepassingen hebben. In dit notebook verkennen we de belangrijkste taken en tonen we hoe verschillende architecturen ingezet worden voor deze problemen.

## Image Classification

**Image classification** is de fundamentele taak van computer vision - het toekennen van een label aan een gehele afbeelding:

### Binaire Classificatie

```python
# Binaire classificatie voorbeeld
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Model voor binaire classificatie (kat vs hond)
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),  # Binaire output
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
```

### Multi-class Classificatie

Voor meer dan twee klassen gebruiken we **softmax activatie**:

$$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$$

## Object Detection

**Object detection** lokaliseert en classificeert objecten in een afbeelding:

### Bounding Box Regressie

Objecten worden vertegenwoordigd door **bounding boxes** met coordinaten:

- **(x, y)**: Centrum van de bounding box
- **w, h**: Breedte en hoogte
- **c**: Confidence score
- **class**: Klasse label

### Two-Stage Detectors

**Faster R-CNN** gebruikt een twee-staps benadering:

1. **Region Proposal Network (RPN)**: Genereert kandidaat regio's
2. **Classification**: Klasseert en verfijnt de regio's

### One-Stage Detectors

**YOLO (You Only Look Once)** behandelt detection als een regressie probleem:

```python
# YOLO output voor elke grid cell
output = [
    p_c,                    # Object confidence
    x, y, w, h,            # Bounding box coordinaten
    p_class1, p_class2,    # Klasse probabilities
    ...
]
```

## Semantic Segmentation

**Semantic segmentation** classificeert elke pixel in de afbeelding:

### Fully Convolutional Networks

FCNs vervangen fully connected lagen door **convolutionele lagen**:

```python
# FCN voor semantic segmentation
class FCN(nn.Module):
    def __init__(self, num_classes):
        super(FCN, self).__init__()
        
        # Encoder (downsampling)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # ... meer lagen
        )
        
        # Decoder (upsampling)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, num_classes, 2, stride=2),
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
```

### U-Net Architectuur

**U-Net** gebruikt skip connections tussen encoder en decoder:

- **Contracting path**: Feature extractie
- **Expansive path**: Lokalisatie met skip connections
- **Final layer**: Pixel-wise classificatie

## Instance Segmentation

**Instance segmentation** combineert object detection met semantic segmentation:

### Mask R-CNN

Mask R-CNN voegt een **mask prediction branch** toe aan Faster R-CNN:

1. **Backbone**: Feature extractie (ResNet, etc.)
2. **RPN**: Region proposals
3. **Box head**: Bounding box regressie en classificatie
4. **Mask head**: Binary mask voor elk gedetecteerd object

## Keypoint Detection

**Keypoint detection** lokaliseert specifieke punten in objecten:

### Human Pose Estimation

Voor menselijke pose detecteren we **17 keypoints**:

- Neus, ogen, oren
- Schouders, ellebogen, polsen
- Heupen, knieën, enkels

### Heatmap Regressie

Keypoints worden voorspeld als **heatmaps**:

```python
# Voor 17 keypoints op 56x56 grid
heatmap = model(image)  # Shape: (batch, 17, 56, 56)
keypoints = argmax_2d(heatmap)  # Extract (x, y) coordinaten
```

## Optical Character Recognition (OCR)

**OCR** combineert tekst detectie met tekst herkenning:

### Text Detection

Verschillende benaderingen:
- **Regression-based**: Voorspel bounding boxes voor tekstregio's
- **Segmentation-based**: Classificeer pixels als tekst/niet-tekst
- **Component-based**: Groepeer connected components

### Text Recognition

Text recognition gebruikt vaak **sequence modeling**:

```python
# CRNN: CNN + RNN voor tekstherkenning
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        
        # Feature extraction
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.ReLU(),
            # ... meer CNN lagen
        )
        
        # Sequence modeling
        self.rnn = nn.LSTM(512, 256, bidirectional=True)
        
        # Transcription
        self.fc = nn.Linear(512, num_classes)
```

## Representation Learning

**Representation learning** leert algemene visuele representaties:

### Self-Supervised Learning

Leren zonder menselijke labels:

- **Contrastive Learning**: Vergelijk positieve en negatieve paren
- **Masked Image Modeling**: Voorspel gemaskeerde beeldregio's
- **Rotation Prediction**: Voorspel rotatiehoek van afbeelding

### Contrastive Learning Framework

```python
# SimCLR: Simple Contrastive Learning
def contrastive_loss(representations, temperature=0.5):
    """
    NT-Xent loss voor contrastive learning
    """
    batch_size = representations.size(0)
    
    # Normaliseer representaties
    representations = F.normalize(representations, dim=1)
    
    # Similarity matrix
    similarity_matrix = torch.matmul(representations, representations.T)
    
    # Mask voor positieve paren
    mask = torch.eye(batch_size, dtype=torch.bool)
    
    # Loss berekening
    positives = similarity_matrix[mask].view(batch_size, -1)
    negatives = similarity_matrix[~mask].view(batch_size, -1)
    
    logits = torch.cat([positives, negatives], dim=1)
    labels = torch.zeros(batch_size, dtype=torch.long)
    
    return F.cross_entropy(logits / temperature, labels)
```

## Taak-Specifieke Uitdagingen

### Class Imbalance

In object detection: achtergrond vs objecten

### Scale Variation

Objecten kunnen sterk verschillen in grootte

### Occlusion

Gedeeltelijk zichtbare objecten

### Multi-label Classification

Afbeeldingen kunnen meerdere labels hebben

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Verschillende taken vergelijken
def compare_vision_tasks():
    """Vergelijk verschillende computer vision taken"""
    
    tasks = {
        'Classification': {
            'output': 'Single label per image',
            'difficulty': 'Low',
            'applications': ['Image search', 'Content filtering'],
            'metrics': ['Accuracy', 'Precision', 'Recall']
        },
        'Object Detection': {
            'output': 'Bounding boxes + labels',
            'difficulty': 'Medium',
            'applications': ['Autonomous driving', 'Surveillance'],
            'metrics': ['mAP', 'IoU', 'Precision']
        },
        'Semantic Segmentation': {
            'output': 'Pixel-wise labels',
            'difficulty': 'High',
            'applications': ['Medical imaging', 'Autonomous driving'],
            'metrics': ['mIoU', 'Pixel accuracy']
        },
        'Instance Segmentation': {
            'output': 'Instance masks + labels',
            'difficulty': 'Very High',
            'applications': ['Medical analysis', 'Robotics'],
            'metrics': ['mAP', 'Mask IoU']
        }
    }
    
    # Visualisatie
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Difficulty comparison
    task_names = list(tasks.keys())
    difficulties = ['Low', 'Medium', 'High', 'Very High']
    diff_values = [difficulties.index(tasks[task]['difficulty']) for task in task_names]
    
    bars = ax1.bar(range(len(task_names)), diff_values, alpha=0.7)
    ax1.set_xlabel('Task')
    ax1.set_ylabel('Difficulty Level')
    ax1.set_title('Computer Vision Tasks: Difficulty Comparison')
    ax1.set_xticks(range(len(task_names)))
    ax1.set_xticklabels(task_names, rotation=45)
    ax1.set_yticks(range(len(difficulties)))
    ax1.set_yticklabels(difficulties)
    
    # Voeg output informatie toe
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                tasks[task_names[i]]['output'],
                ha='center', va='bottom', rotation=45, fontsize=8)
    
    # Applications word cloud style
    ax2.axis('off')
    ax2.set_title('Applications by Task')
    
    y_pos = 0.8
    for task, info in tasks.items():
        ax2.text(0.1, y_pos, f"{task}:", fontweight='bold')
        for app in info['applications']:
            y_pos -= 0.1
            ax2.text(0.2, y_pos, f"• {app}")
        y_pos -= 0.1
    
    plt.tight_layout()
    plt.show()
    
    return tasks

# Vergelijk taken
tasks = compare_vision_tasks()

print("\nGedetailleerde Task Vergelijking:")
for task, info in tasks.items():
    print(f"\n{task}:")
    print(f"  Output: {info['output']}")
    print(f"  Difficulty: {info['difficulty']}")
    print(f"  Applications: {', '.join(info['applications'])}")
    print(f"  Metrics: {', '.join(info['metrics'])}")

## Praktische Voorbeelden per Taak

Laten we kijken naar concrete implementaties voor verschillende taken:

In [ ]:
# Voorbeeld implementaties voor verschillende taken

# 1. Image Classification Model
class ImageClassifier(nn.Module):
    """Model voor image classification"""
    def __init__(self, num_classes=10):
        super(ImageClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 2. Simple Object Detector (conceptueel)
class SimpleObjectDetector(nn.Module):
    """Conceptueel model voor object detection"""
    def __init__(self, num_classes=20, num_boxes=2, grid_size=7):
        super(SimpleObjectDetector, self).__init__()
        self.grid_size = grid_size
        
        # Feature extractor
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # ... meer lagen
        )
        
        # Detection head: voor elke grid cell voorspel
        # [p_c, x, y, w, h, p_class1, p_class2, ...]
        output_channels = num_boxes * (5 + num_classes)  # 5 = p_c, x, y, w, h
        self.detector = nn.Conv2d(64, output_channels, 1)
    
    def forward(self, x):
        x = self.features(x)
        x = self.detector(x)
        # Reshape naar (batch, grid, grid, num_boxes * (5 + num_classes))
        x = x.permute(0, 2, 3, 1)
        return x

# 3. Simple Segmentation Model
class SimpleSegmentationModel(nn.Module):
    """Model voor semantic segmentation"""
    def __init__(self, num_classes=21):
        super(SimpleSegmentationModel, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, 2, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, num_classes, 1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Model vergelijking
models = {
    'Classification': ImageClassifier(),
    'Object Detection': SimpleObjectDetector(),
    'Segmentation': SimpleSegmentationModel()
}

print("Model Output Shapes Vergelijking:")
dummy_input = torch.randn(1, 3, 224, 224)

for name, model in models.items():
    with torch.no_grad():
        output = model(dummy_input)
    print(f"{name}: {output.shape}")
    
    # Bereken aantal parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"  Parameters: {total_params:,}")
    print()